In [1]:
import pandas as pd
import os
import numpy as np
import datetime
import json
os.getcwd()

'/home/jian/Projects/SunnyD/Allocate_DMA_spend_to_IRI/Update_2020_Joann_Request'

In [2]:
Excel_File=pd.ExcelFile("./SUNNYD - Digital Deliveries by Day by Market_0113.xlsx")
Excel_File.sheet_names

['YouTube',
 'summary',
 'total cost',
 'DTV',
 'DTV cost',
 'DTV total imp',
 'ct-need to cleanDMA',
 'untitled 3cost',
 'untitled 7im',
 'ALLeee',
 'Snap',
 'data',
 'mapping_naming',
 'coefficient',
 'Digital Clicks',
 'SUNNYD - Digital Spend by Day b']

In [3]:
'''
# update dma naming

DMA_name_mapping=json.load(open("/home/jian/Docs/Geo_mapping/dma_map_latest.json","r"))
print(len(DMA_name_mapping))
joann_updating=Excel_File.parse('mapping_naming',dtype=str)
joann_updating=joann_updating[~joann_updating['dirty_name'].isin(DMA_name_mapping.keys())]
joann_updating_dict=joann_updating.set_index("dirty_name").to_dict()['clean_name']

print(len(joann_updating_dict))
DMA_name_mapping.update(joann_updating_dict)
print(len(DMA_name_mapping))

with open("/home/jian/Docs/Geo_mapping/dma_map_latest.json", 'w') as outfile:  
    json.dump(DMA_name_mapping, outfile)
'''

# 1612
# 108
# 1720

'\n# update dma naming\n\nDMA_name_mapping=json.load(open("/home/jian/Docs/Geo_mapping/dma_map_latest.json","r"))\nprint(len(DMA_name_mapping))\njoann_updating=Excel_File.parse(\'mapping_naming\',dtype=str)\njoann_updating=joann_updating[~joann_updating[\'dirty_name\'].isin(DMA_name_mapping.keys())]\njoann_updating_dict=joann_updating.set_index("dirty_name").to_dict()[\'clean_name\']\n\nprint(len(joann_updating_dict))\nDMA_name_mapping.update(joann_updating_dict)\nprint(len(DMA_name_mapping))\n\nwith open("/home/jian/Docs/Geo_mapping/dma_map_latest.json", \'w\') as outfile:  \n    json.dump(DMA_name_mapping, outfile)\n'

In [4]:
DMA_name_mapping=json.load(open("/home/jian/Docs/Geo_mapping/dma_map_latest.json","r"))

In [5]:
df_data=Excel_File.parse('data',dtype=str)
df_data['cleaned date']=pd.to_datetime(df_data['cleaned date']).dt.date
df_data['Impression']=df_data['Impression'].fillna(0).astype(float)
df_data['Click']=df_data['Click'].fillna(0).astype(int)
df_data['Cost']=df_data['Cost'].fillna(0).astype(float)
print(df_data.shape)
df_data.head(2)

(53494, 6)


,Media,clean_DMA,Impression,Click,Cost,cleaned date
0,Hulu,"Abilene-Sweetwater, TX",0.0,0,0.0,2019-10-18
1,Hulu,"Albany-Schenectady-Troy, NY",0.0,0,0.0,2019-10-18


In [6]:
# Clean the Snapchat data based on new file
df_data_snap=df_data[df_data['Media']=="Snapchat"]
df_data=df_data[df_data['Media']!="Snapchat"]

df_snapchat_new=pd.read_csv("./SunnyD Fall FY'20 Snap_Spend by DMA_1.16.20.csv",dtype=str,
                            usecols=['cleaned DMA','Impressions','Swipe Ups','Spend'])
df_snapchat_new['Impressions']=df_snapchat_new['Impressions'].astype(int)
df_snapchat_new['Swipe Ups']=df_snapchat_new['Swipe Ups'].astype(int)
df_snapchat_new['Spend']=df_snapchat_new['Spend'].astype(float)


df_snapchat_new=df_snapchat_new.rename(columns={"Impressions":"new_impr","Swipe Ups":"new_click","Spend":"new_cost"})
df_snapchat_new=df_snapchat_new.groupby("cleaned DMA")['new_impr','new_click','new_cost'].sum().reset_index()
df_snapchat_new['impr_pctg']=df_snapchat_new['new_impr']/df_snapchat_new['new_impr'].sum()
df_snapchat_new['click_pctg']=df_snapchat_new['new_click']/df_snapchat_new['new_click'].sum()
df_snapchat_new['cost_pctg']=df_snapchat_new['new_cost']/df_snapchat_new['new_cost'].sum()


In [7]:
df_snapchat_ratio=df_snapchat_new[['cleaned DMA','impr_pctg','click_pctg','cost_pctg']]
df_snapchat_ratio['clean_DMA']=df_snapchat_ratio['cleaned DMA'].copy()
df_snapchat_ratio['merge_key']=1


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
del df_data_snap['clean_DMA']
df_data_snap['merge_key']=1
print(df_data_snap['Impression'].sum())
print(df_data_snap['Click'].sum())
print(df_data_snap['Cost'].sum())

df_data_snap=pd.merge(df_data_snap,df_snapchat_ratio,on="merge_key",how="left")
df_data_snap['new_impr_by_dma']=df_data_snap['Impression']*df_data_snap['impr_pctg']
df_data_snap['new_click_by_dma']=df_data_snap['Click']*df_data_snap['click_pctg']
df_data_snap['new_cost_by_dma']=df_data_snap['Cost']*df_data_snap['cost_pctg']

del df_data_snap['Impression']
del df_data_snap['Click']
del df_data_snap['Cost']

df_data_snap=df_data_snap.rename(columns={"new_impr_by_dma":"Impression","new_click_by_dma":"Click","new_cost_by_dma":"Cost"})

print(df_data_snap['Impression'].sum())
print(df_data_snap['Click'].sum())
print(df_data_snap['Cost'].sum())


df_data_snap=df_data_snap[df_data.columns.tolist()]
df_data=df_data.append(df_data_snap)

# Snapchat already updated

18352805.0
112417
42081.38999999999
18352805.0
112417.0
42081.39


In [9]:
df_data_snap.head(2)

,Media,clean_DMA,Impression,Click,Cost,cleaned date
0,Snapchat,"Albany, GA",96.576694,0.619329,0.150699,2019-10-22
1,Snapchat,"Albany-Schenectady-Troy, NY",86.450250,0.287546,0.121902,2019-10-22


In [10]:
df_data.head(2)

,Media,clean_DMA,Impression,Click,Cost,cleaned date
0,Hulu,"Abilene-Sweetwater, TX",0.0,0.0,0.0,2019-10-18
1,Hulu,"Albany-Schenectady-Troy, NY",0.0,0.0,0.0,2019-10-18


In [11]:
df_buy_type=Excel_File.parse('summary',dtype=str)
df_buy_type=df_buy_type[['Media','Media buy']].drop_duplicates().rename(columns={"Media buy":"buy_type"})
df_data=pd.merge(df_data,df_buy_type,on="Media",how="left")
print(df_data.shape)

(56893, 7)


In [12]:
print(df_data[pd.isnull(df_data['buy_type'])]['Media'].unique().tolist())

del df_data['buy_type']
df_data['Media']=df_data['Media'].replace("Microsegment Pre-Roll TTD","Microsegment Video/Display")
df_data['Media']=df_data['Media'].replace("Pandora","Pandora/Soundcloud")
df_data['Media']=df_data['Media'].replace("Facebook","Facebook/Instagram")
df_data['Media']=df_data['Media'].replace("DTV","DirecTV")
df_data=pd.merge(df_data,df_buy_type,on="Media",how="left")
print(df_data.shape)

print(df_data['buy_type'].unique().tolist())
df_data.head(2)

['Microsegment Pre-Roll TTD', 'Pandora', 'Facebook', 'DTV']
(56893, 7)
['National', 'Zip', 'DMA']


,Media,clean_DMA,Impression,Click,Cost,cleaned date,buy_type
0,Hulu,"Abilene-Sweetwater, TX",0.0,0.0,0.0,2019-10-18,National
1,Hulu,"Albany-Schenectady-Troy, NY",0.0,0.0,0.0,2019-10-18,National


In [13]:
df_net_to_gross_factor=Excel_File.parse("coefficient")
df_net_to_gross_factor['Media']=df_net_to_gross_factor['Media'].replace("DTV","DirecTV")
df_net_to_gross_factor['Media']=df_net_to_gross_factor['Media'].replace("Microsegment Pre-Roll TTD","Microsegment Video/Display")
df_net_to_gross_factor['Media']=df_net_to_gross_factor['Media'].replace("Pandora","Pandora/Soundcloud")
df_net_to_gross_factor['Media']=df_net_to_gross_factor['Media'].replace("Facebook","Facebook/Instagram")


df_net_to_gross_factor=df_net_to_gross_factor[['Media','coefficient']]
df_net_to_gross_factor=pd.merge(df_net_to_gross_factor,df_buy_type,on="Media",how="left")

In [14]:
# 2019 mapping 

allocation_pctg=pd.read_csv("/home/jian/Projects/SunnyD/Otherinput/DMA_to_IRI_Pctg/from_DMA_to_IRI_based_on_common_population.csv",dtype=str)
allocation_pctg=allocation_pctg[['DMA','IRI Market','Pctg']]
allocation_pctg['Pctg']=allocation_pctg['Pctg'].apply(lambda x: float(x))
allocation_pctg['clean_DMA']=allocation_pctg['DMA'].apply(lambda x: DMA_name_mapping[x])
allocation_pctg=allocation_pctg[['clean_DMA','IRI Market','Pctg']]
allocation_pctg

,clean_DMA,IRI Market,Pctg
0,"Abilene-Sweetwater, TX",NO MARKET,1.000000
1,"Albany, GA",NO MARKET,1.000000
2,"Albany-Schenectady-Troy, NY",ALBANY,0.796801
3,"Albany-Schenectady-Troy, NY",NEW ENGLAND,0.029085
4,"Albany-Schenectady-Troy, NY",NO MARKET,0.174114
...,...,...,...
322,"Wilmington, NC",NO MARKET,1.000000
323,"Yakima-Pasco-Richland-Kennewick, WA",NO MARKET,1.000000
324,"Youngstown, OH",NO MARKET,1.000000
325,"Yuma, AZ-El Centro, CA",NO MARKET,1.000000


In [15]:
# update_national_pctg

df_pop_by_iri=pd.read_csv("/home/jian/Projects/SunnyD/Otherinput/Population_by_IRI/SunnyD_population by IRI with duplication_JL_2018-09-21.csv")
df_pop_by_iri.head(2)

df_pop_by_iri=df_pop_by_iri.groupby("IRI Market")['POP18'].sum().to_frame().reset_index()
total_pop=df_pop_by_iri['POP18'].sum() # with dup
print("total_pop: ",total_pop)

df_pop_by_iri['Pctg']=df_pop_by_iri['POP18']/total_pop
df_pop_by_iri.head(2)
# df_pop_by_iri['Pctg'].sum()
df_pop_by_iri['clean_DMA']="xx"
df_pop_by_iri=df_pop_by_iri[['clean_DMA','IRI Market','Pctg']]

allocation_pctg=allocation_pctg.append(df_pop_by_iri)


# update AK HI DMAs

df_ak_hi=pd.DataFrame({"clean_DMA":['Anchorage, AK', 'Fairbanks, AK', 'Honolulu, HI', 'Juneau, AK']})
df_ak_hi['IRI Market']="NO MARKET"
df_ak_hi['Pctg']=1

allocation_pctg=allocation_pctg.append(df_ak_hi)
allocation_pctg

total_pop:  339192928.3000005


,clean_DMA,IRI Market,Pctg
0,"Abilene-Sweetwater, TX",NO MARKET,1.000000
1,"Albany, GA",NO MARKET,1.000000
2,"Albany-Schenectady-Troy, NY",ALBANY,0.796801
3,"Albany-Schenectady-Troy, NY",NEW ENGLAND,0.029085
4,"Albany-Schenectady-Troy, NY",NO MARKET,0.174114
...,...,...,...
64,xx,WICHITA,0.002469
0,"Anchorage, AK",NO MARKET,1.000000
1,"Fairbanks, AK",NO MARKET,1.000000
2,"Honolulu, HI",NO MARKET,1.000000


In [16]:
df_data[df_data['clean_DMA']=="xx"]['Media'].unique()

array(['Hulu', 'InMarket', 'Microsegment Video/Display',
       'Pandora/Soundcloud', 'Twitch', 'YouTube', 'Facebook/Instagram'],
      dtype=object)

In [17]:
# 1. National buy
df_1=df_data[df_data['buy_type']=="National"]
print(df_1.shape)
print(df_1['Media'].unique())

print("df_1['Impression'].sum()",df_1['Impression'].sum())
print("df_1['Click'].sum()",df_1['Click'].sum())
print("df_1['Cost'].sum()",df_1['Cost'].sum())

# Anyway need to allocate all 211 DMAs
'''
df_1=df_1[df_1[['Impression','Click','Cost']].sum(axis=1)>0]
print(df_1.shape)
'''
df_1=pd.merge(df_1,allocation_pctg,on="clean_DMA",how="left")
df_1['IRI_Impression']=df_1['Impression']*df_1['Pctg']
df_1['IRI_Click']=df_1['Click']*df_1['Pctg']
df_1['IRI_Cost']=df_1['Cost']*df_1['Pctg']
df_1=df_1.groupby(["IRI Market","Media","buy_type","cleaned date"])['IRI_Impression','IRI_Click','IRI_Cost'].sum().reset_index()

print("/n")
print("df_1['IRI_Impression'].sum()",df_1['IRI_Impression'].sum())
print("df_1['IRI_Click'].sum()",df_1['IRI_Click'].sum())
print("df_1['IRI_Cost'].sum()",df_1['IRI_Cost'].sum())

(10745, 7)
['Hulu' 'DirecTV']
df_1['Impression'].sum() 50255409.0
df_1['Click'].sum() 0.0
df_1['Cost'].sum() 505573.99
/n
df_1['IRI_Impression'].sum() 50255409.00000001
df_1['IRI_Click'].sum() 0.0
df_1['IRI_Cost'].sum() 505573.99


In [18]:
# 2. DMA buy
df_2=df_data[df_data['buy_type']=="DMA"]
print(df_2.shape)
print(df_2['Media'].unique())

print("df_2['Impression'].sum()",df_2['Impression'].sum())
print("df_2['Click'].sum()",df_2['Click'].sum())
print("df_2['Cost'].sum()",df_2['Cost'].sum())

df_2=pd.merge(df_2,allocation_pctg,on="clean_DMA",how="left")
df_2['IRI_Impression']=df_2['Impression']*df_2['Pctg']
df_2['IRI_Click']=df_2['Click']*df_2['Pctg']
df_2['IRI_Cost']=df_2['Cost']*df_2['Pctg']
df_2=df_2.groupby(["IRI Market","Media","buy_type","cleaned date"])['IRI_Impression','IRI_Click','IRI_Cost'].sum().reset_index()

print("After/n")
print("df_2['IRI_Impression'].sum()",df_2['IRI_Impression'].sum())
print("df_2['IRI_Click'].sum()",df_2['IRI_Click'].sum())
print("df_2['IRI_Cost'].sum()",df_2['IRI_Cost'].sum())

(18624, 7)
['Pandora/Soundcloud' 'Twitch' 'Snapchat']
df_2['Impression'].sum() 26898779.0
df_2['Click'].sum() 138184.0
df_2['Cost'].sum() 262178.57
After/n
df_2['IRI_Impression'].sum() 26898779.0
df_2['IRI_Click'].sum() 138184.0
df_2['IRI_Cost'].sum() 262178.57000000007


In [19]:
# 3. Zip buy


# Step1: get the mapping without "NO MARKET"
df_3=df_data[df_data['buy_type']=="Zip"]
print(df_3.shape)
print(df_3['Media'].unique())

print("df_3['Impression'].sum()",df_3['Impression'].sum())
print("df_3['Click'].sum()",df_3['Click'].sum())
print("df_3['Cost'].sum()",df_3['Cost'].sum())

df_3=pd.merge(df_3,allocation_pctg,on="clean_DMA",how="left")

# remove if No Market, and rolling to 100% for those in IRI
df_3_DMA_IRI=df_3[['clean_DMA','IRI Market','Pctg']].drop_duplicates()
df_3_DMA_IRI=df_3_DMA_IRI[df_3_DMA_IRI['IRI Market']!="NO MARKET"]
df_3_DMA_IRI_pair_adj=df_3_DMA_IRI.groupby(['clean_DMA'])['Pctg'].sum().to_frame().reset_index()
df_3_DMA_IRI_pair_adj['adj']=1/df_3_DMA_IRI_pair_adj['Pctg']
df_3_DMA_IRI_pair_adj=df_3_DMA_IRI_pair_adj[['clean_DMA','adj']]
df_3_DMA_IRI=pd.merge(df_3_DMA_IRI,df_3_DMA_IRI_pair_adj,on="clean_DMA",how="left")
df_3_DMA_IRI['Pctg']=df_3_DMA_IRI['Pctg']*df_3_DMA_IRI['adj']

del df_3_DMA_IRI['adj']

df_3_DMA_IRI

# Step2: use the new mapping table
df_3=df_data[df_data['buy_type']=="Zip"]
print(df_3.shape)

print("df_3['Impression'].sum()",df_3['Impression'].sum())
print("df_3['Click'].sum()",df_3['Click'].sum())
print("df_3['Cost'].sum()",df_3['Cost'].sum())

df_3=pd.merge(df_3,df_3_DMA_IRI,on="clean_DMA",how="left")

# Step3: even though zip buy, but soem DMAs shown with 0 overlap with DMA, have to fill na with "NO MARKET"
df_3['IRI Market']=df_3['IRI Market'].fillna("NO MARKET")
df_3['Pctg']=df_3['Pctg'].fillna(1)


df_3['IRI_Impression']=df_3['Impression']*df_3['Pctg']
df_3['IRI_Click']=df_3['Click']*df_3['Pctg']
df_3['IRI_Cost']=df_3['Cost']*df_3['Pctg']
df_3=df_3.groupby(["IRI Market","Media","buy_type","cleaned date"])['IRI_Impression','IRI_Click','IRI_Cost'].sum().reset_index()

print("After/n")
print("df_3['IRI_Impression'].sum()",df_3['IRI_Impression'].sum())
print("df_3['IRI_Click'].sum()",df_3['IRI_Click'].sum())
print("df_3['IRI_Cost'].sum()",df_3['IRI_Cost'].sum())

(27524, 7)
['InMarket' 'Microsegment Video/Display' 'YouTube' 'Facebook/Instagram']
df_3['Impression'].sum() 67745643.0
df_3['Click'].sum() 114068.0
df_3['Cost'].sum() 347375.010000178
(27524, 7)
df_3['Impression'].sum() 67745643.0
df_3['Click'].sum() 114068.0
df_3['Cost'].sum() 347375.010000178
After/n
df_3['IRI_Impression'].sum() 67745642.99999999
df_3['IRI_Click'].sum() 114068.0
df_3['IRI_Cost'].sum() 347375.010000178


In [33]:
df_output=pd.concat([df_1,df_2,df_3])
print(df_output.shape)
df_output=pd.merge(df_output,df_net_to_gross_factor,on=["Media",'buy_type'],how="left")
print(df_output.shape)

df_output['IRI_Gross_Cost']=df_output['IRI_Cost']*df_output['coefficient']
df_output.to_csv("./SunnyD_allocated_DMA_to_IRI_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


(18424, 7)
(18424, 8)
